In [80]:
import pandas as pd

ModuleNotFoundError: No module named 'chroms'

In [83]:
# read the UCSC_genes_chr21.txt file into data frame
chr21_known_genes = pd.read_csv('UCSC_genes_chr21_all', sep='\t')
# remove the part of columns names before the last dot
chr21_known_genes.columns = chr21_known_genes.columns.str.split('.').str[-1]
chr21_known_genes['geneSymbol'] = chr21_known_genes['geneSymbol'].astype(str)
chr21_known_genes['refseq'] = chr21_known_genes['refseq'].astype(str)
chr21_known_genes["strand"] = chr21_known_genes["strand"].str.strip()
chr21_known_genes.head()

,name,chrom,strand,txStart,txEnd,geneSymbol,refseq
0,uc002zkg.1,chr21,+,9690070,9690100,DQ579288,nan
1,uc021wgt.1,chr21,+,9692177,9692207,DQ590589,nan
2,uc011abu.2,chr21,+,9711934,9769227,BC071797,nan
3,uc021wgu.1,chr21,+,9825459,9825481,JB074932,nan
4,uc021wgv.1,chr21,+,9825831,9826011,MIR3648,nan


In [70]:
# read HK_genes.txt file into data frame, with column names as GeneSymbol, GeneID
HK_genes = pd.read_csv('HK_genes.txt', sep='\t', header=None, names=['geneSymbol', 'refseq'])
HK_genes['geneSymbol'] = HK_genes['geneSymbol'].astype(str)
HK_genes['refseq'] = HK_genes['refseq'].astype(str)
# trim space at the end of geneSymbol column in HK_genes data frame
HK_genes['geneSymbol'] = HK_genes['geneSymbol'].str.strip()
HK_genes.head()

,geneSymbol,refseq
0,AAAS,NM_015665
1,AAGAB,NM_024666
2,AAMP,NM_001087
3,AAR2,NM_015511
4,AARS,NM_001605


In [84]:
# merge the two data frames if they match on geneSymbol or refseq
column_to_check_dups = ['chrom', 'txStart', 'txEnd']
merge_on_refseq = pd.merge(chr21_known_genes, HK_genes, on='refseq', how='inner')
merge_on_genesymbol = pd.merge(chr21_known_genes, HK_genes, on='geneSymbol', how='inner')
# Combining the results
df_merged = pd.concat([merge_on_refseq, merge_on_genesymbol], ignore_index=True).drop_duplicates(subset=column_to_check_dups, keep='first')

df_merged

,name,chrom,strand,txStart,txEnd,geneSymbol_x,refseq,geneSymbol_y,geneSymbol,refseq_x,refseq_y
0,uc002yjx.2,chr21,-,16333555,16437126,NRIP1,NM_003489,NRIP1,NaN,NaN,NaN
1,uc031ruy.1,chr21,-,16333555,16438231,NRIP1,NM_003489,NRIP1,NaN,NaN,NaN
3,uc021whl.1,chr21,-,16337036,16340513,NRIP1,NM_003489,NRIP1,NaN,NaN,NaN
4,uc002yjy.1,chr21,+,17102495,17252377,USP25,NM_013396,USP25,NaN,NaN,NaN
8,uc002yls.3,chr21,-,27096790,27107965,ATP5PF,NM_001685,ATP5J,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
135,uc002zdz.3,chr21,+,45432205,45480110,NaN,NaN,NaN,TRAPPC10,nan,NM_003274
139,uc011afb.1,chr21,+,45509365,45518940,NaN,NaN,NaN,TRAPPC10,nan,NM_003274
146,uc002zij.3,chr21,-,47608359,47648738,NaN,NaN,NaN,LSS,NM_001001438,NM_002340
150,uc002zio.1,chr21,-,47655047,47667074,NaN,NaN,NaN,MCM3AP,nan,NM_003906


In [89]:
def calc_promoter(df, before, after):
    if df["strand"] == "+":
        df['promoter_start'] = df['txStart'] - before
        df['promoter_end'] = df['txStart'] + after
    elif df["strand"] == "-":
        df['promoter_start'] = df['txEnd'] - after
        df['promoter_end'] = df['txEnd'] + before
    return df

In [90]:
# calculate promoter start and end for each gene
df_merged = df_merged.apply(calc_promoter, axis=1, args=(5000, 1000))
df_merged.head()

,name,chrom,strand,txStart,txEnd,geneSymbol_x,refseq,geneSymbol_y,geneSymbol,refseq_x,refseq_y,promoter_start,promoter_end
0,uc002yjx.2,chr21,-,16333555,16437126,NRIP1,NM_003489,NRIP1,NaN,NaN,NaN,16436126,16442126
1,uc031ruy.1,chr21,-,16333555,16438231,NRIP1,NM_003489,NRIP1,NaN,NaN,NaN,16437231,16443231
3,uc021whl.1,chr21,-,16337036,16340513,NRIP1,NM_003489,NRIP1,NaN,NaN,NaN,16339513,16345513
4,uc002yjy.1,chr21,+,17102495,17252377,USP25,NM_013396,USP25,NaN,NaN,NaN,17097495,17103495
8,uc002yls.3,chr21,-,27096790,27107965,ATP5PF,NM_001685,ATP5J,NaN,NaN,NaN,27106965,27112965
